In [106]:
import sys

In [110]:
import collections
from queue import PriorityQueue

In [111]:
file = open('chicken.txt', 'r')
text = file.read()
file.close()
text

# Stop if file is empty
if len(text) == 0:
    print("empty file. Stopping")
    sys.exit(0)

In [112]:
characters = list(text)
uniq_chars = set(characters)

In [113]:
uniq_chars

{'\n', 'F', 'a', 'e', 'm', 'r', 's'}

In [114]:
freq = collections.Counter(characters)
freq

Counter({'F': 1, 'a': 1, 'r': 2, 'm': 1, 'e': 1, 's': 1, '\n': 1})

In [115]:
class HuffNode:
    def __init__(self, character, weight, leftNode=None, rightNode=None):
        self.character = character
        self.weight = weight
        self.leftNode = leftNode
        self.rightNode = rightNode
        
        # We keep all the huffman codes of the children in self.codes
        if leftNode is None:
            # leaf node
            self.codes = [(character, '')]
        else:
            # prepend 0 to all codes in the left subtree and keep in codes
            self.codes = [(x[0], '0'+x[1]) for x in leftNode.codes]
            # prepend 1 to all codes in the right subtree and also keep in codes
            self.codes += [(x[0], '1'+x[1]) for x in rightNode.codes]
    
    def __lt__(self, other):
        return True
    

In [116]:
# Initialize a priority queue
q = PriorityQueue()

# Push each character & its frequency to the queue as HuffNodes
for c,f in freq.items():
    # print(c, f)
    node = HuffNode(c, f)
    q.put((f, node))
    
    
# Huffman Code generation
# Combine lowest frequency nodes iteratively
rootNode = None
while(q.empty() is False):
    # if there is only one element in the queue stop
    if q.qsize() < 2:
        rootNode = q.get()[1]
        break
        
    # combine two lowest frequency nodes to one node
    node1 = q.get()[1]
    node2 = q.get()[1]
    node = HuffNode(character='', weight=node1.weight+node2.weight, leftNode=node1, rightNode=node2)
    # print(node.weight)    
    # insert back to the queue until the root node is reached
    q.put((node.weight, node))


In [117]:
rootNode.codes

[('\n', '000'),
 ('e', '001'),
 ('r', '01'),
 ('a', '100'),
 ('s', '101'),
 ('m', '110'),
 ('F', '111')]

In [118]:
# Convert codes into a dictionary/map
code_map = dict(rootNode.codes)


In [119]:
code_map



{'\n': '000',
 'e': '001',
 'r': '01',
 'a': '100',
 's': '101',
 'm': '110',
 'F': '111'}

In [120]:
code_map.get('\n')


'000'

In [121]:
# Generate the compressed text
compressed_list = [code_map.get(c) for c in characters]

# A function to convert binary bits to bytes
# input: '1011' output: \x0b (which is 00001011 in binary: a single byte/8 bits)
# Basically the function splits the sequence of bits into chunks of 8-bits
# Refer: https://stackoverflow.com/questions/51425638/how-to-write-huffman-coding-to-a-file-using-python
def _to_bytes(data):
    b = bytearray()
    for i in range(0, len(data), 8):
        b.append(int(data[i:i+8], 2))
    return bytes(b)

In [123]:
compressed_text = ''.join(compressed_list)
# compressed_text = '101'
print(compressed_text)
# Ideally number of bits should be a multiple of 8
# otherwise keep track of how many off bits to recover back while decoding
offset = len(compressed_text) % 8


1111000111000101101000


In [124]:
offset

6

In [65]:
# We have to save the characters & their corresponding huffman codes also
# This is required for decoding
code_block = ''
for k in code_map.keys():
    code_block += k + ':' + code_map.get(k)

# Convert the code block into bytes for saving in a binary file    
code_block_bytes = bytes(code_block, encoding='utf-8')
code_block_offset = len(code_block_bytes)


In [125]:
code_block = ''
for k in code_map.keys():
    code_block += k

code_block += ':'    
    
for k in code_map.keys():
    code_block += code_map.get(k) + ','
    
# Convert the code block into bytes for saving in a binary file    
code_block_bytes = bytes(code_block, encoding='utf-8')
code_block_offset = len(code_block_bytes)


In [154]:
code_block

'\nerasmF:000,001,01,100,101,110,111,'

In [155]:
# Write the encoded bytes
with open('chicken.bin', 'wb') as fbout:
    # First write the code_block_offset to determine how to bytes to be read
    # for retrieving the code block safely
    b = format(code_block_offset, '016b') # two bytes = 16 bits
    fbout.write(_to_bytes(b))
    # Now write the code block as its ascii (utf-8 encoding): a couple of bytes
    fbout.write(code_block_bytes)
    
    # then we write the number of off bits; this is the number of bits in the last byte
    fbout.write(bytes([offset]))
    # then convert the bit sequence to a byte sequence
    compressed_bytes = _to_bytes(compressed_text)
    fbout.write(compressed_bytes)


In [156]:
offset

6

In [157]:
# Decoding: Read the binary file
with open('chicken.bin', 'rb') as fbin:
    encoded_bytes = fbin.read()


In [191]:
# Decoding: first two bytes gives the code_block_offset
a1 = encoded_bytes[0]
a2 = encoded_bytes[1]
code_block_offset = a1 * 256 + a2
# The code block is stores from index 2 to code_block_offset+2
code_block = encoded_bytes[2: code_block_offset+2].decode('utf-8')
print(code_block)



erasmF:000,001,01,100,101,110,111,


In [193]:
2**16

65536

In [182]:
a = format(5403, '016b')
b = a[:8]
c = a[8:]

In [183]:
int(b, 2)

21

In [185]:
int(b,2)*256 + int(c, 2)

5403

In [146]:
bytes([256])

ValueError: bytes must be in range(0, 256)

In [152]:
b = format(256, '016b')
_to_bytes(b)

b'\x01\x00'

In [153]:
b

'0000000100000000'

In [133]:
code_block.split(':')

['\nerasmF', '000,001,01,100,101,110,111,']

In [139]:
## Get back the code_map from the code_block
code_map = {}
keys = code_block.split(':')[0]
codes = code_block.split(':')[1]

i = 0
for code in codes.split(','):
    if len(code) > 0:
        code_map[code] = keys[i]
        i += 1

In [140]:
code_map

{'000': '\n',
 '001': 'e',
 '01': 'r',
 '100': 'a',
 '101': 's',
 '110': 'm',
 '111': 'F'}

In [138]:
## Get back the code_map from the code_block (Actually reverse code_map)
code_map = {}
for code in code_block.split('\n'):
    if (len(code)>1):
        character = code.split(':')[0]
        huffcode = code.split(':')[1]
        code_map[huffcode] = character

In [141]:
# Decoding: Remember the next byte is the offset value of the last byte
# We split the encoded_bytes to get just the code sequence alone
code_bytes = encoded_bytes[code_block_offset+1:]
offset = code_bytes[0]

encoded_text = ''
# Read from the next byte till the second last byte
for ebyte in code_bytes[1:len(code_bytes)-1]:
    b = format(ebyte, '08b') # in 8-bit format
    encoded_text += b
    
# The last byte may not have used full 8 bits; remove un-necessary zeros with the offset value
last_byte = code_bytes[len(code_bytes)-1]
formatter = '0' + str(offset) + 'b'
encoded_text += format(last_byte, formatter)

print(encoded_text)


1111000111000101101000


In [142]:
len(encoded_text) % 8

6

In [143]:
# Decoding: Now decode the retireved binary sequence with the reverse code_map
# This is a greedy algorithm
def decoding(content, _lookup):
    while content:
        options = [i for i in _lookup if content.startswith(i) and (any(content[len(i):].startswith(b) for b in _lookup) or not content[len(i):])]
        if not options:
            raise Exception("Decoding error")
        yield _lookup[options[0]]
        content = content[len(options[0]):]

print(''.join(decoding(encoded_text, code_map)))

Farmers



In [331]:
b = bytearray('aBc', encoding='utf-8')
b[0]


97

In [362]:
bits = '0000000110'
int(bits[::-1], 2).to_bytes(2, 'little')


b'\x80\x01'

In [367]:
b = '10111111'
hex(int(b[::-1], 2))



'0xfd'

In [406]:
bits = "11000010"
print(len(bits))
_to_bytes(bits)

8


b'\xc2'

In [423]:
n = bytes([5])

In [427]:
n[0]

5

In [414]:
n = bytes('5', 'utf-8')

In [415]:
n.decode('utf-8')

'5'

In [418]:
ord('1')

49

In [270]:
blist = list('111')
blist

['1', '1', '1']

In [272]:
import functools
# e.g. 111 -> 7 (given a binary string, convert it into decimal)
def decimal_from_binary_string(bstring):
    blist = list(bstring)
    return functools.reduce(lambda a,b : int(a)<<1 | int(b), blist)
    

In [278]:
decimal_from_binary_string('00')

0

In [405]:
int('111', 2)

7

In [528]:
from tkinter import *
from tkinter.ttk import *
  
# importing askopenfile function 
# from class filedialog 
from tkinter.filedialog import askopenfile 


root = Tk()
# Code to add widgets will go here...
root.geometry('800x600')

# frame inside root window 
frame = Frame(root)   
# geometry method 
frame.pack()  
# button inside frame which is  
# inside root 
button = Button(frame, text ='Choose File')   
button.pack()       

root.mainloop()





In [24]:
from tkinter import *
from tkinter.ttk import *
  
# importing askopenfile function 
# from class filedialog 
from tkinter.filedialog import askopenfile 

root = Tk() 
root.geometry('600x400') 
root.title('Hufman Coding')
  
# This function will be used to open 
# file in read mode and only Python files 
# will be opened 
def open_file_encode(): 
    file = askopenfile(mode ='r', filetypes =[('Text', '*.txt')])
    if file:
        print(file.name)
        
def open_file_decode(): 
    file = askopenfile(mode ='r', filetypes =[('Binary', '*.bin')])
    if file:
        print(file.name)

button_frame = Frame(root, height=100, width=600, borderwidth=1)
button_frame.pack()
btn1 = Button(button_frame, text ='Encode File', command = open_file_encode) 
btn2 = Button(button_frame, text ='Decode File', command = open_file_decode)

#btn1.pack(side = TOP, pady = 10) 
btn1.grid(row=0, column=0, padx=50, pady=10)
btn2.grid(row=0, column=1, padx=50, pady=10)

text_frame = Frame(root, height=200, width=600, borderwidth = 2)
text_frame.pack()
text_widget = Text(text_frame, height = 200, width = 600)
text_widget.pack()
text_widget.insert(END, 'A bunch of text can be inserted here')
  
mainloop() 


/Users/bnjasim/Desktop/Upwork/huffman/testfile.txt


In [15]:
file.name

NameError: name 'file' is not defined

In [29]:
with open('huffmanGUI.py', 'r') as f:
    text = f.read()
    newtext = text.replace('\t', '    ')

In [30]:
with open('huffmanGUI.py', 'w') as f:
    f.write(newtext)